# Collaborative Filtering

In [1]:
import os
import sys

sys.path.append("/Users/eric/Code/recommender-system/src")

import csv
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

from data.db import Recommendations
from data.db import insert

NB_USERS = 200000
NB_ITEMS = 300000

In [2]:
X = sparse.lil_matrix((NB_USERS, NB_ITEMS))

In [3]:
%%time

with open("../datasets/movielens/ml-20m/ratings.csv", "r") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader, None)  # skip header

    for i, (user_id, movie_id, rating, timestamp) in enumerate(reader):
        X[user_id, movie_id] = rating

CPU times: user 49min 2s, sys: 34.3 s, total: 49min 36s
Wall time: 52min 1s


### Cosine Similarity on Items

In [4]:
%time item_similarities = cosine_similarity(X.T)

CPU times: user 1min 25s, sys: 32 s, total: 1min 57s
Wall time: 2min 15s


In [5]:
%%time

to_insert = []

for i in range(len(item_similarities)):
    for index, score in enumerate(item_similarities[i]):
        if score < 0.5 or i == index:
            continue
            
        r = Recommendations(**{"engine_name": "CosineMovies",
                               "source_product_id": i,
                               "recommended_product_id": index,
                               "score": score})
    
        to_insert.append(r)
        
    if i == 10000:
        break

CPU times: user 16min 19s, sys: 20.2 s, total: 16min 39s
Wall time: 17min 18s


In [6]:
len(to_insert)

7364

In [7]:
insert(to_insert)